# Exemplo de calculadora usando o PLY (Python Lex-Yacc)

## Importando o ply

In [18]:
from ply.lex import lex
from ply.yacc import yacc

Faremos uma calculadora para a seguinte gramatica:

S' -> EXPRESSAO <br>
EXPRESSAO -> numero OPERACAO numero <br>
OPERACAO -> mais <br>
OPERACAO -> menos <br>
OPERACAO -> multiplicacao <br>
OPERACAO -> divisao <br>

## Analisador Lexico (Lex)

O analisador lexico define os tokens que serão usados na linguagem. A definição de Token para o nosso contexto são os terminais da gramática. Os tokens são definidos por duas coisas: A primeira é a uma lista ou tupla com o nome de variavel chamado `tokens`, a segunda é com nomes de variaveis ou por funções do tipo `t_NOME_TOKEN`, mas entre nome de variaveis e funções, independentemente da forma que escolhermos, devemos liga-los a uma expressão regular que define o padrão de cada token. Um detalhe importante é que **a ordem de definição dos tokens (t_NOME_TOKEN) é importante, ele sempre dará preferencia aos tokens definidos primeiro**, por isso é importante definir os tokens com menos expressividade primeiro. A baixo temos os tokens mais basicos que usaremos para o nosso exemplo de calculadora.

In [19]:

# nome dos tokens de operadores e constantes
reservados = ('mais','menos','multiplicacao','divisao') 

# expressões regulares para tokens de operadores e constantes 

t_mais = r'\+'
t_menos = r'-'
t_multiplicacao = r'\*'
t_divisao = r'/'

reservados

('mais', 'menos', 'multiplicacao', 'divisao')

Aqui em baixo vamos definir numeros, como este é o nosso token de maior expressividade temos que defini-lo por ultimo

In [20]:

def t_numero(t): # aqui definimos o token numero, ele nesse caso converte o valor direto para um inteiro, mas poderia ser um float
    r'\d+'
    t.value = int(t.value)
    return t

t_ignore = ' \t\n' # ignora espaços e tabs

def t_error(t): # nos dizer qual caractere ilegal e se tem erro
    print("Caracter ilegal: ", t.value[0])
    t.lexer.skip(1)

tokens = reservados + ('numero',)
tokens

('mais', 'menos', 'multiplicacao', 'divisao', 'numero')

Agora vamos instanciar o nosso analisador lexico, no inicio vamos deixa-lo com o modo de debug ativado

In [21]:
__file__ = 'calculadora.ipynb' # somente para funcionar no jupyter notebook

lexer = lex(debug=True) # construção do lexer

lex: tokens   = ('mais', 'menos', 'multiplicacao', 'divisao', 'numero')
lex: literals = ''
lex: states   = {'INITIAL': 'inclusive'}
lex: Adding rule t_numero -> '\d+' (state 'INITIAL')
lex: Adding rule t_mais -> '\+' (state 'INITIAL')
lex: Adding rule t_multiplicacao -> '\*' (state 'INITIAL')
lex: Adding rule t_menos -> '-' (state 'INITIAL')
lex: Adding rule t_divisao -> '/' (state 'INITIAL')
lex: ==== MASTER REGEXS FOLLOW ====
lex: state 'INITIAL' : regex[0] = '(?P<t_numero>\d+)|(?P<t_mais>\+)|(?P<t_multiplicacao>\*)|(?P<t_menos>-)|(?P<t_divisao>/)'


Com o debug em true, podemos ver que ele mostra como ele internamente guarda os tokens (como `<t_nome_token>`), podemos ver tambem as expressões regulares assosiadas a ele.

Antes de continuarmos, vale a pena relembrar que o PLY nos disponibiliza outras ferramentas como `literals`, aqui somente estará o minimo necessario para fazer o trabalho.

## Analisador Sintatico (Yacc)

### Gramatica de Atributos

Gramatica de atributos é uma materia que está bem no final do curso, então se você esta lendo isso e não saber o que é, expliquei brevemente o que é e como funciona. Caso o contrario, pode pular para a proxima seção.


S → S MAIS A <br>
S → A <br>
A → 1 <br>
A → 0 <br>

Se conectar funções a essa gramatica ficaria algo assim (ligação esta sendo representado por ⇛)

S → S MAIS A ⇛ `lambda S,MAIS,A: S + A` <br>
S → A ⇛ `lambda A: A` <br>
A → 1 ⇛ `lambda: 1` <br>
A → 0 ⇛ `lambda: 0` <br>

Então se executarmos a gramatica de atributos para a entrada `1+1+1+0` teriamos o output igual a 3

### Criando a gramatica de atributos

Para o PLY a gramatica é definida por funções com `p_REGRA_DA_GRAMATICA`, essas funções precisam de uma regra associada a ela, esta e criada a partir da `DEFINIÇÃO DE DOCUMENTAÇÃO DE FUNÇÃO DO PYTHON que se define entre 3 aspas no INICIO da função`, essas funções tem como parametro um array que a partir do index 1 são os tokens e o retorno de outras regras, o primeiro valor desse array (index 0) é o retorno da regra atual, para ficar mais claro, vamos ver um exemplo:

```py
def p_INICIO(regras):
    '''
    INICIO : EXPRESSAO
           | numero
    '''
    regras[0] = regras[1]
```

No caso do codigo acima, ele é equivalente a dizermos:

S → EXPRESSAO <br>
S → num <br>

Então agora vamos analisar a regra abaixo

In [22]:

def p_EXPRESSAO(regras):
    '''
    EXPRESSAO : numero mais numero
              | numero menos numero
              | numero multiplicacao numero
              | numero divisao numero
              | numero
    '''
    if len(regras) == 2: # checa se é o caso de um número ou uma constante
        regras[0] = regras[1]
        
    else: # checa se é o caso de uma operação
        if regras[2] == '+':
            regras[0] = regras[1] + regras[3]
        elif regras[2] == '-':
            regras[0] = regras[1] - regras[3]
        elif regras[2] == '*':
            regras[0] = regras[1] * regras[3]
        elif regras[2] == '/':
            regras[0] = regras[1] / regras[3]

Podemos ver que essa função ficou um pouco grande e com muitas regras, aconselho tomar um tempo lendo ela para entender bem como funciona a gramatica de atributos.

Ao longo do trabalho vocês, desenvolvedores do analisador, poderam criar novas regras, inclusive para diminuir regras grandes, então eu abaixo nos quebraremos essa regra em regras menores.

In [23]:
def p_EXPRESSAO(regras):
    '''
    EXPRESSAO : numero OPERACAO numero
    '''
    
    regras[0] = regras[2](regras[1], regras[3])
    

def p_OPERACAO(regras):
    '''
    OPERACAO : mais
             | menos
             | multiplicacao
             | divisao
    '''
    if regras[1] == '+':
        regras[0] = lambda x,y: x+y
    elif regras[1] == '-':
        regras[0] = lambda x,y: x-y
    elif regras[1] == '*':
        regras[0] = lambda x,y: x*y
    elif regras[1] == '/':
        regras[0] = lambda x,y: x/y
        


Pronto agora temos mais regras, porem elas estão mais simples e mais faceis de entender, agora vamos tentar rodar nosso analisador lexico e sintatico juntos.

In [24]:
def p_error(regras):
    print("Erro de sintaxe"+ str(regras))

parser = yacc(debug=True) # construção do parser

AttributeError: module 'parsetab' has no attribute '_tabversion'

Pronto temos um analisadore lexico e sintatico, agora vamos ver se ele consegue avaliar a expressão `3 * 5`.

PS: Ele ta dando esses WARNINGS porque ele não precisa daquele INICIO que definimos no inicio, ele ja gera internamente um estado S, pode-se ver isso atraves do parser.out, que é um arquivo de debug do Yacc.

In [8]:
parser.parse('23 + 4')

27